Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [287]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [288]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [289]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [290]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [291]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [292]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    sigma=(0.5,0.5)
    row = [r for r, c in enumerate(state.rows) if c > 0][random.gauss(0,sigma)]
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [293]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    print(cooked)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


In [294]:
from numpy import argsort


def possible_moves(state: Nim):
    return [Nimply(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]

def fitness(state:Nim):
    return -nim_sum(state)

def ES(state: Nim, mu=10, l=50, n_iter=100):
    best, best_eval = None, float("inf")
    n_children = int(l / mu)

    # Population
    population=[]
    for r,c in enumerate(state.rows):
        if c>=1:
            population.append(Nimply(r,random.randint(1,c)))
    
    # perform the search
    for epoch in range(n_iter):
        # evaluate fitness for the population
        scores=[]
        for c in population:
            tmp = deepcopy(state)
            tmp.nimming(c)
            scores.append(fitness(tmp))
        ranks = argsort(argsort(scores))
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
        # create children from parents
        children = list()
        for i in selected:
			# check if this parent is the best solution ever seen
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                #print('%d, Best: f(%s) = %.5f' % (epoch, best, best_eval))
			# create children for parent
            for _ in range(n_children):
                child = None
                while child is None:
                    child = Nimply(population[i][0],random.randint(1,state.rows[population[i][0]]))
                    children.append(child)
		# replace population with children
        population = children
    return best

## Oversimplified match

In [297]:
logging.getLogger().setLevel(logging.INFO)

strategy = (pure_random, ES)

nim = Nim(10)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won with {strategy[player]}!")

INFO:root:init : <1 3 5 7 9 11 13 15 17 19>
INFO:root:ply: player 0 plays Nimply(row=5, num_objects=11)
INFO:root:status: <1 3 5 7 9 0 13 15 17 19>
INFO:root:ply: player 1 plays Nimply(row=9, num_objects=14)
INFO:root:status: <1 3 5 7 9 0 13 15 17 5>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=4)
INFO:root:status: <1 3 5 3 9 0 13 15 17 5>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=4)
INFO:root:status: <1 3 1 3 9 0 13 15 17 5>
INFO:root:ply: player 0 plays Nimply(row=9, num_objects=5)
INFO:root:status: <1 3 1 3 9 0 13 15 17 0>
INFO:root:ply: player 1 plays Nimply(row=7, num_objects=5)
INFO:root:status: <1 3 1 3 9 0 13 10 17 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=1)
INFO:root:status: <1 3 0 3 9 0 13 10 17 0>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 0 3 9 0 13 10 17 0>
INFO:root:ply: player 0 plays Nimply(row=7, num_objects=3)
INFO:root:status: <0 3 0 3 9 0 13 7 17 0>
INFO:root:ply: player 1 plays Nimply